In [ ]:
from gmail_agent import gmail_agent
from calendar_agent import Calendar_agent

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition,InjectedState
from langchain_core.messages import (
    HumanMessage,
    ToolMessage,
    
)
from langgraph.types import Command
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools.base import InjectedToolCallId


from typing_extensions import TypedDict
from typing import Annotated

import os
from dotenv import load_dotenv 
from token_creator import get_creds
#get graph visuals
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod
import os

import gradio as gr


In [2]:
load_dotenv()
GOOGLE_API_KEY=os.getenv('google_api_key')

In [3]:
GEMINI_MODEL='gemini-2.0-flash'
llm = ChatGoogleGenerativeAI(google_api_key=GOOGLE_API_KEY, model=GEMINI_MODEL, temperature=0.3)

In [4]:
mail_agent=gmail_agent(llm)
calendar_agent=Calendar_agent(llm)

In [5]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [6]:
@tool
def mail_manager(query:str):
    """Tool to use to answer any email related queries
    this tool can:
    refresh the inbox
    show the inbox
    get a specific mail's content to display
    create a draft of a new mail
    show the current draft
    send the draft
    list all the drafts
    args: query - pass the email related queries directly here
    """
    response=mail_agent.chat(str(query))
    return response

In [ ]:
@tool
def calendar_manager(query:str):
    """tool to use to answere any calendar or schedule related queries
    this tool can:
    create recuring events
    create quick events
    refresh the calendar
    show the calendar
    args: query - pass the calendar related queries directly here
    """
    response=calendar_agent.chat(str(query))
    return response

In [8]:
class google_agent:
    def __init__(self,llm: any):
        self.agent=self._setup(llm)
    def _setup(self,llm):
   
        langgraph_tools=[mail_manager,calendar_manager]



        graph_builder = StateGraph(State)

        # Modification: tell the LLM which tools it can call
        llm_with_tools = llm.bind_tools(langgraph_tools)
        tool_node = ToolNode(tools=langgraph_tools)
        def chatbot(state: State):
            """ travel assistant that answers user questions about their trip.
            Depending on the request, leverage which tools to use if necessary."""
            return {"messages": [llm_with_tools.invoke(state['messages'])]}

        graph_builder.add_node("chatbot", chatbot)

        
        graph_builder.add_node("tools", tool_node)
        # Any time a tool is called, we return to the chatbot to decide the next step
        graph_builder.add_edge(START,'chatbot')
        graph_builder.add_edge("tools", "chatbot")
        graph_builder.add_conditional_edges(
            "chatbot",
            tools_condition,
        )
        memory=MemorySaver()
        graph=graph_builder.compile(checkpointer=memory)
        return graph
        

    def display_graph(self):
        return display(
                        Image(
                                self.agent.get_graph().draw_mermaid_png(
                                    draw_method=MermaidDrawMethod.API,
                                )
                            )
                        )
    def stream(self,input:str):
        config = {"configurable": {"thread_id": "1"}}
        input_message = HumanMessage(content=input)
        for event in self.agent.stream({"messages": [input_message]}, config, stream_mode="values"):
            event["messages"][-1].pretty_print()

    def chat(self,input:str):
        config = {"configurable": {"thread_id": "1"}}
        response=self.agent.invoke({'messages':HumanMessage(content=str(input))},config)
        return response['messages'][-1].content
    
    def get_state(self, state_val:str):
        config = {"configurable": {"thread_id": "1"}}
        return self.agent.get_state(config).values[state_val]

In [9]:
agent=google_agent(llm)

In [12]:
mail_agent.get_state('messages')

[HumanMessage(content='show the inbox', additional_kwargs={}, response_metadata={}, id='ce193593-a6b2-414e-b87e-9b4bc7b1de22'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'show_inbox', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-3552e797-8882-4813-a4f9-e413f854cdea-0', tool_calls=[{'name': 'show_inbox', 'args': {}, 'id': '34993166-5018-4b25-839b-90def2265277', 'type': 'tool_call'}], usage_metadata={'input_tokens': 220, 'output_tokens': 3, 'total_tokens': 223, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{\'1953b17a61a8b129\': {\'From\': \'LinkedIn <messages-noreply@linkedin.com>\', \'To\': \'tristan padiou <padioutristan@gmail.com>\', \'Date\': \'Tue, 25 Feb 2025 01:10:52 +0000 (UTC)\', \'label\': [\'UNREAD\', \'CATEGORY_SOCIAL\', \'INBOX\'], \'subject\': \'Learning Spotlight: Build Real-World Skills\', \'Snippet\': \'Learn by D

In [ ]:
# agent.stream('list the drafts')

================================ Human Message =================================

what are some events in my calendar
================================== Ai Message ==================================
Tool Calls:
  calendar_manager (b3e6da10-be68-47d5-9668-2c42cf06941f)
 Call ID: b3e6da10-be68-47d5-9668-2c42cf06941f
  Args:
    query: show the calendar
================================= Tool Message =================================
Name: calendar_manager

OK. I see the following events in your calendar:

- Titouan: Festival, ile de re on 2025-09-07T11:00:00-04:00
- Loanne birthday on 2025-12-14, 2026-12-14, 2027-12-14, 2028-12-14, 2029-12-14, 2030-12-14, 2031-12-14, 2032-12-14, 2033-12-14, 2034-12-14
- Anna birthday on 2025-12-21, 2026-12-21, 2027-12-21, 2028-12-21, 2029-12-21, 2030-12-21, 2031-12-21, 2032-12-21, 2033-12-21
================================== Ai Message ==================================

OK. I see the following events in your calendar:

- Titouan: Festival, ile de re on 

In [ ]:
def chatbot(input,history):
    response=agent.chat(input)
    return response


In [ ]:
demo = gr.ChatInterface(chatbot, type="messages", autofocus=False)

if __name__ == "__main__":
    demo.launch()